In [1]:
!git clone https://github.com/amira20nasser/Forum-Discussions-Categorization.git


fatal: destination path 'Forum-Discussions-Categorization' already exists and is not an empty directory.


In [2]:
!pip install dask_ml
!pip install emoji
!pip install nltk
!pip install dask_ml

In [3]:
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
import re
import emoji
import nltk
import numpy as np
import dask.dataframe as dd
from nltk import ngrams
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from urllib.parse import urlparse
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from dask_ml.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
dataset_path = 'Forum-Discussions-Categorization/nn-25-forum-discussions-categorization'
train = dd.read_csv(dataset_path + '/train.csv')
train=train.sample(frac=0.5, random_state=42)
X = train['Discussion']
y = train['Category']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=42)


/usr/local/lib/python3.10/dist-packages/dask_ml/model_selection/_split.py:464: FutureWarning: The default value for 'shuffle' must be specified when splitting DataFrames. In the future DataFrames will automatically be shuffled within blocks prior to splitting. Specify 'shuffle=True' to adopt the future behavior now, or 'shuffle=False' to retain the previous behavior.
  warnings.warn(


# Text Preprocessing

In [6]:



def extract_url_features(url):
    parsed_url = urlparse(url)
    domain = parsed_url.netloc
    path = parsed_url.path
    query = parsed_url.query
    tld = domain.split('.')[-1]
    domain = re.sub(r'\bwww\b', '', domain)
    domain = re.sub(r'\.com\b', '', domain)
    return  domain+' '+path

def place_holder_url(text):
    if not isinstance(text, str):
        return ""
    urls = re.findall(r'https?://(?:www\.)?[^\s/$.?#].[^\s]*', text)
    for url in urls:
        text = text.replace(url, extract_url_features(url))
    return text

def remove_urls(text):
    if not isinstance(text, str):
        return ""
    return re.sub(r'https?://(?:www\.)?[^\s/$.?#].[^\s]*', '', text)


def clean_text(text):
    if not isinstance(text, str):
        return " "
    text = text.lower()
    text = re.sub(r'(.)\1+', r'\1', text) #repetitive oooh ahhh
    text = re.sub(r'\s+', ' ', text)

    text_no_emojis = emoji.replace_emoji(text, replace=' ')
    text_no_numbers = re.sub(r'\d+', ' ', text_no_emojis)

    return text_no_numbers

def preprocess(text):
    text = place_holder_url(text)
    text = clean_text(text)
    # Tokenize using BERT tokenizer
    # and return input_ids instead of word embeddings
    return text





In [7]:
# Reprocess your data with the updated preprocess function
X_train = X_train.apply(preprocess, meta=('Discussion', 'object')).compute()
X_valid = X_valid.apply(preprocess, meta=('Discussion', 'object')).compute()

## Encoding Y

In [8]:
def encode_labels(labels):
  manual_mapping = {
      'Politics': 0,
      'Sports': 1,
      'Media': 2,
      'Market & Economy': 3,
      'STEM': 4
  }
  encoded_labels = [manual_mapping[label] for label in labels]
  return encoded_labels

In [9]:
y_train = encode_labels(y_train.compute())
y_valid = encode_labels(y_valid.compute())
y_train[0:2]

[4, 1]

In [10]:
y_train = np.array(y_train)
y_valid = np.array(y_valid)

In [11]:
y_train

array([4, 1, 4, ..., 0, 0, 0])

#   **BERT**

In [12]:
import tensorflow as tf

# Pad sequences to make them the same length
#X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, padding='post', maxlen=128)
#X_valid = tf.keras.preprocessing.sequence.pad_sequences(X_valid, padding='post', maxlen=128)

In [13]:
X_train=list(X_train)
X_valid=list(X_valid)

In [14]:
# X_train = X_train.tolist()
# X_valid = X_valid.tolist()


In [15]:
! pip install transformers
from transformers import AutoTokenizer
from transformers import TFAutoModel


/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [16]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [17]:
encoded_X_train=list(range(len(X_train)))
encoded_X_train = tokenizer(
    X_train,
    add_special_tokens=True,
    max_length=128,
    padding="max_length",
    truncation=True,
    return_tensors="np"
)

X_input_ids = encoded_X_train["input_ids"]
X_attention_masks = encoded_X_train["attention_mask"]

In [18]:
encoded_X_valid = tokenizer(
    X_valid,
    add_special_tokens=True,
    max_length=128,
    padding="max_length",
    truncation=True,
    return_tensors="np"
)
X_valid_input_ids = encoded_X_valid["input_ids"]
X_valid_attention_masks = encoded_X_valid["attention_mask"]

In [19]:
X_input_ids

array([[  101,  1996,  5056, ...,     0,     0,     0],
       [  101,  2012,  2777, ...,     0,     0,     0],
       [  101, 29583,  2003, ...,     0,     0,     0],
       ...,
       [  101,  8529,  1010, ...,     0,     0,     0],
       [  101,  1045,  2333, ...,     0,     0,     0],
       [  101,  4058,   999, ...,     0,     0,     0]])

In [20]:
X_attention_masks.shape


(11204, 128)

# Positional embedding

In [ ]:
import numpy as np
def get_pos_encoding_matrix(max_position,d_model, n=10000):
    position = np.arange(max_position) # generates [0,1,2,...,max_pos]
    embedding=np.arange(d_model)//2 # generates [0,0,1,1,...,d_model]
    freqs = n**(2*embedding/d_model)
    pos_enc = position.reshape(-1,1)*freqs.reshape(1,-1)
    pos_enc[:, ::2] = np.sin(pos_enc[:, ::2])
    pos_enc[:, 1::2] = np.cos(pos_enc[:, 1::2])
    return pos_enc
# POS = get_pos_encoding_matrix(seq_len,hidden_size)

In [21]:
transformer_model = TFAutoModel.from_pretrained('distilbert-base-uncased')


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [22]:
batch_size = 32  
total_samples = X_input_ids.shape[0]
hidden_size = 768  
seq_len = X_input_ids.shape[1]

X_embeddings = np.zeros((total_samples, seq_len, hidden_size))

# Processing the whole data crashes due to memory constraints, so we will process in batches of 32 samples
for start_idx in range(0, total_samples, batch_size):
    end_idx = min(start_idx + batch_size, total_samples)
    batch_input_ids = X_input_ids[start_idx:end_idx]
    batch_attention_masks = X_attention_masks[start_idx:end_idx]
    
    batch_output = transformer_model(batch_input_ids, attention_mask=batch_attention_masks).last_hidden_state
    
    X_embeddings[start_idx:end_idx] = batch_output.numpy() # + POS

total_samples = X_valid_input_ids.shape[0]
X_valid_embeddings = np.zeros((total_samples, seq_len, hidden_size))

for start_idx in range(0, total_samples, batch_size):
    end_idx = min(start_idx + batch_size, total_samples)
    batch_input_ids = X_valid_input_ids[start_idx:end_idx]
    batch_attention_masks = X_valid_attention_masks[start_idx:end_idx]
    
    batch_output = transformer_model(batch_input_ids, attention_mask=batch_attention_masks).last_hidden_state
    
    X_valid_embeddings[start_idx:end_idx] = batch_output.numpy() # + POS



In [23]:
X_embeddings[0].shape

(128, 768)

In [24]:
X_valid_embeddings[0].shape

(128, 768)

In [25]:
X_valid_embeddings

array([[[-0.0252765 ,  0.06586355,  0.03916127, ...,  0.03105751,
          0.47747347,  0.32698092],
        [-0.05776632,  0.07927404, -0.07407765, ...,  0.22486848,
          0.52158099,  0.19623813],
        [-0.05895076,  0.36125568,  0.37599421, ...,  0.02272255,
          0.37893316,  0.1830285 ],
        ...,
        [ 0.04978791,  0.23749354,  0.26954299, ...,  0.18898943,
          0.03557941, -0.01399909],
        [ 0.04617302,  0.25647879,  0.38225645, ...,  0.24241517,
         -0.09111279, -0.15216722],
        [ 0.07392971,  0.21142869,  0.27393419, ...,  0.26004124,
         -0.06132312, -0.16935758]],

       [[-0.45035529, -0.35580668, -0.3469713 , ..., -0.27404499,
          0.35337454,  0.26601994],
        [-0.57584059,  0.13661504, -0.33406371, ..., -0.21729708,
          0.15921444,  0.25057498],
        [-0.09335721,  0.050469  , -0.41342187, ..., -0.12656078,
          0.12664047, -0.12126939],
        ...,
        [-0.03673357,  0.08808765,  0.09505028, ..., -

In [26]:
#from tensorflow.keras import mixed_precision
#mixed_precision.set_global_policy('mixed_float16')

In [27]:
import tensorflow as tf
from tensorflow.keras.regularizers import L2
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
from keras.optimizers import Adam
with strategy.scope():
    from tensorflow.keras.models import Model
    from tensorflow.keras.layers import Dense, MultiHeadAttention, LayerNormalization,GlobalAveragePooling1D, Dropout, Embedding, Flatten,Input, Add
    from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
    
    initializer='he_normal'
    
    def transformer_block(x):
        attention_output=MultiHeadAttention(num_heads=2,key_dim=hidden_size,kernel_initializer=initializer, kernel_regularizer=L2(1e-3))(x,x,x)
        attention_output=Dropout(0.1)(attention_output)
        x = Add()([x, attention_output])  
        x = LayerNormalization(epsilon=1e-6)(x)
        feed_forward_output=Dense(128,activation='gelu',kernel_initializer=initializer,kernel_regularizer=L2(1e-3))(x)
        x = Dropout(0.1)(x)
        x=Dense(128,activation='gelu',kernel_initializer=initializer,kernel_regularizer=L2(1e-3))(feed_forward_output)
        x = Dropout(0.1)(x)
        x = Add()([x, feed_forward_output])  
        output = LayerNormalization(epsilon=1e-6)(x)
        return output
    
    input = Input(shape=(seq_len,hidden_size), name='input', dtype='float32')
    x = LayerNormalization(epsilon=1e-6)(input)
    x = Dense(256, activation='linear',kernel_initializer=initializer,kernel_regularizer=L2(1e-3))(x) # reduce embedding dimension to 256
    x=transformer_block(x)
    #x=transformer_block(x)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x=Dense(64,activation='gelu',kernel_initializer=initializer,kernel_regularizer=L2(1e-3))(x)
    x = Dropout(0.1)(x)
    x=Dense(32,activation='gelu',kernel_initializer=initializer,kernel_regularizer=L2(1e-3))(x)
    x = Dropout(0.1)(x)
    output_layer=Dense(5,activation='softmax',kernel_initializer='glorot_uniform',kernel_regularizer=L2(1e-3))(x)
    model=Model(inputs=input,outputs=output_layer)
    model.summary()
    adam = Adam(learning_rate=0.00001,
        clipnorm=1.0,)
    model.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


Number of devices: 2


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)        │ (None, 128, 768)       │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization       │ (None, 128, 768)       │          1,536 │ input[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128, 256)       │        196,864 │ layer_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention      │ (None, 128, 256)       │      1,577,728 │ dense[0][0],           │
│ (MultiHeadAttention)      │                        │                │ dense[0][0],           │
│                           │                        │                │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 128, 256)       │              0 │ multi_head_attention[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 128, 256)       │              0 │ dense[0][0],           │
│                           │                        │                │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_1     │ (None, 128, 256)       │            512 │ add[0][0]              │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 128, 128)       │         32,896 │ layer_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 128, 128)       │         16,512 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_3 (Dropout)       │ (None, 128, 128)       │              0 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_1 (Add)               │ (None, 128, 128)       │              0 │ dropout_3[0][0],       │
│                           │                        │                │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_2     │ (None, 128, 128)       │            256 │ add_1[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d  │ (None, 128)            │              0 │ layer_normalization_2… │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 64)             │          8,256 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_4 (Dropout)       │ (None, 64)             │              0 │ dense_3[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)      

 Total params: 1,836,805 (7.01 MB)

 Trainable params: 1,836,805 (7.01 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:
early_stopping = EarlyStopping(monitor= 'val_loss',patience=3)
checkpoint = ModelCheckpoint('model.keras', 
                    monitor="val_loss", mode="min", 
                    save_best_only=True, verbose=1)
history=model.fit(X_embeddings, y_train, batch_size=4, epochs=50, callbacks=[early_stopping,checkpoint], validation_data=(X_valid_embeddings, y_valid))
model.load_weights('model.keras')


Epoch 1/50
2800/2801 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.4332 - loss: 7.6815
Epoch 1: val_loss improved from inf to 7.05251, saving model to model.keras
2801/2801 ━━━━━━━━━━━━━━━━━━━━ 79s 22ms/step - accuracy: 0.4333 - loss: 7.6813 - val_accuracy: 0.6698 - val_loss: 7.0525
Epoch 2/50
2801/2801 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6537 - loss: 7.0918
Epoch 2: val_loss improved from 7.05251 to 6.80469, saving model to model.keras
2801/2801 ━━━━━━━━━━━━━━━━━━━━ 60s 21ms/step - accuracy: 0.6537 - loss: 7.0917 - val_accuracy: 0.6977 - val_loss: 6.8047
Epoch 3/50
2801/2801 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.7014 - loss: 6.7792
Epoch 3: val_loss improved from 6.80469 to 6.49229, saving model to model.keras
2801/2801 ━━━━━━━━━━━━━━━━━━━━ 60s 21ms/step - accuracy: 0.7014 - loss: 6.7791 - val_accuracy: 0.7256 - val_loss: 6.4923
Epoch 4/50
2800/2801 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.7053 - loss: 6.5140
Epoch 4: val_loss improved from 6.49229 to 6.

In [29]:
loss, accuracy = model.evaluate(X_valid_embeddings, y_valid)
print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")

41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.7049 - loss: 1.7458
Validation Loss: 1.7457314729690552, Validation Accuracy: 0.7317829728126526


# Testing

In [30]:
import keras
from keras.models import load_model


In [31]:
loaded_model = load_model("model.keras")

In [32]:
import dask.dataframe as dd

# Preprocessing text data
dataset_path = 'Forum-Discussions-Categorization/nn-25-forum-discussions-categorization'
test = dd.read_csv(dataset_path + '/test.csv')
X_test=test['Discussion']
X_test = X_test.apply(preprocess, meta=('Discussion', 'object')).compute()
X_test=list(X_test)
encoded_X_test=tokenizer(
    X_test,
    add_special_tokens=True,
    max_length=128,
    padding="max_length",
    truncation=True,
    return_tensors="np"
)
X_test_input_ids = encoded_X_test["input_ids"]
X_test_attention_masks = encoded_X_test["attention_mask"]

X_test_embeddings=get_embeddings(X_test_input_ids,X_test_attention_masks)

In [ ]:
y_test=loaded_model.predict(X_test_embeddings)
predictions=np.argmax(y_test, axis=1)

In [ ]:
import pandas as pd
output=pd.DataFrame(predictions)
output.index +=1
output.index.name='SampleID'
output.columns = ['Discussion']
output.to_csv('output.csv', index=True)